In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn.metrics import classification_report

In [63]:
df = pd.read_csv(r"C:\Users\C J HARINI\Downloads\Bank_Personal_Loan_Modelling.csv")

In [64]:
df

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,29,3,40,92697,1,1.9,3,0,0,0,0,1,0
4996,4997,30,4,15,92037,4,0.4,1,85,0,0,0,1,0
4997,4998,63,39,24,93023,2,0.3,3,0,0,0,0,0,0
4998,4999,65,40,49,90034,3,0.5,2,0,0,0,0,1,0


In [65]:
df.drop(["ID", "ZIP Code"],axis=1,inplace=True)

In [66]:
print(df["Experience"].unique())
df["Experience"] = abs(df["Experience"])

[ 1 19 15  9  8 13 27 24 10 39  5 23 32 41 30 14 18 21 28 31 11 16 20 35
  6 25  7 12 26 37 17  2 36 29  3 22 -1 34  0 38 40 33  4 -2 42 -3 43]


In [67]:
df.columns

Index(['Age', 'Experience', 'Income', 'Family', 'CCAvg', 'Education',
       'Mortgage', 'Personal Loan', 'Securities Account', 'CD Account',
       'Online', 'CreditCard'],
      dtype='object')

In [68]:
df = df[['Age', 'Experience', 'Income', 'Family', 'CCAvg','Education', 'Mortgage', 'Securities Account','CD Account', 'Online', 'CreditCard', 'Personal Loan']]




In [69]:
X = df.iloc[:,:-1].values
Y = df.iloc[:,-1].values

In [70]:
##SPLITING TRAIN AND TEST DATA

In [71]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
x,y = X,Y
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25,random_state=69)
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)
x_train.shape, y_train.shape

((3750, 11), (3750,))

In [72]:
##PYTORCH

In [73]:
import torch
from torch.utils.data import DataLoader, TensorDataset

In [74]:
BATCH_SIZE = 1
LEARNING_RATE = 0.005
EPOCH = 30

In [75]:
train_x = torch.from_numpy(x_train).to(torch.float32)
train_y = torch.from_numpy(y_train).to(torch.float32)
train_x.shape, train_y.shape

(torch.Size([3750, 11]), torch.Size([3750]))

In [76]:
data = TensorDataset(train_x,train_y)
data = DataLoader(data,batch_size=BATCH_SIZE,shuffle=True)

In [77]:
##MODEL

In [78]:
class Model(torch.nn.Module):
    
    def __init__(self):
        super(Model,self).__init__()
        
        self.layer1 = torch.nn.Linear(11,16)
        self.layer2 = torch.nn.Linear(16,1)
        self.sigmoid = torch.nn.Sigmoid()
        self.relu = torch.nn.ReLU()
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        x = self.sigmoid(x)
        return x

In [79]:
model = Model()
print(model)

Model(
  (layer1): Linear(in_features=11, out_features=16, bias=True)
  (layer2): Linear(in_features=16, out_features=1, bias=True)
  (sigmoid): Sigmoid()
  (relu): ReLU()
)


In [80]:
##GENETIC ALGORITHM

In [81]:
BATCH_SIZE = 1
data = TensorDataset(train_x,train_y)
data = DataLoader(data,batch_size=BATCH_SIZE,shuffle=True)

In [82]:
def fitness_function(model):
    
    y_pred = model(train_x)
    y_pred = torch.where(y_pred>=0.5, 1, 0).flatten()
    accuracy = (y_pred == train_y).sum().float().item() / len(data.dataset)
    
    return accuracy

In [83]:
def crossover_mutation(model1,model2):
    shp = [i.numpy().shape for i in model1.parameters()]
    sz = [i[0]*i[1] if len(i) == 2 else i[0] for i in shp]

    matrix_1 = np.concatenate([i.numpy().flatten() for i in model1.parameters()])
    matrix_2 = np.concatenate([i.numpy().flatten() for i in model2.parameters()])
    
    start = len(matrix_1)//2 - 10
    end = len(matrix_1)//2 + 10
    mid_value = random.randrange(start,end)
    
    main_child_1 = np.concatenate([matrix_1[:mid_value], matrix_2[mid_value:]])
    main_child_2 = np.concatenate([matrix_2[:mid_value], matrix_1[mid_value:]])
    
    random_start = random.randrange(0,len(matrix_1)//2)
    random_end = random.randrange(random_start,len(matrix_1))
    
    child_1_mutate = main_child_1.copy()
    child_1_mutate[random_start:random_end] = child_1_mutate[random_start:random_end][::-1]
    
    random_start = random.randrange(0,len(matrix_1)//2)
    random_end = random.randrange(random_start,len(matrix_1))
    
    child_2_mutate = main_child_2.copy()
    child_2_mutate[random_start:random_end] = child_2_mutate[random_start:random_end][::-1]
    children = [main_child_1, main_child_2, child_1_mutate, child_2_mutate]
    output = list()
    
    for child in children:
        param = list()
        cum_sum = 0
        for i in range(len(sz)):
            array = child[cum_sum : cum_sum + sz[i]]
            array = array.reshape(shp[i])
            cum_sum += sz[i]
            param.append(array)
        param = np.array(param, dtype="object")
        output.append(param)
    
    output = np.array(output, dtype="object")
    return output

In [84]:
torch.manual_seed(69)
torch.set_grad_enabled(False)
population_size = 10

def train(no):
    population = np.array([Model() for i in range(population_size)])
    
    best_model = None
    
    for loop in range(no):
        # Sort population based up on fitness function
        population = population[np.argsort([fitness_function(model) for model in population])]
        
        # Printing Max Accuracy
        best_model = population[-1]
        if(loop%10 == 0):
            print("Gen", loop, " :", fitness_function(population[-1]))
        
        # Parent Selection
        parent_max_1 = population[-1]
        parent_max_2 = population[-2]
        parent_max_3 = population[-3]
        parent_max_4 = population[-4] 
        parent_1 = population[0]
        parent_2 = population[1]
        
        # Crossover
        output_1 = crossover_mutation(parent_max_1, parent_max_3)
        output_2 = crossover_mutation(parent_max_2, parent_max_4)
        output = np.concatenate([output_1, output_2])
        
        # New population
        new_population = np.array([Model() for i in range(len(output))])
        for count, model in enumerate(new_population, 0):
            for index, param in enumerate(model.parameters(), 0):
                param.data = (torch.tensor(output[count][index]))
        
        # Add Poor performing parent to the new population
        new_population = np.concatenate([new_population, [parent_1, parent_2]])
        
        # New population becomes your next population
        population = new_population.copy()
        
    return best_model

In [85]:
best = train(100)

Gen 0  : 0.824
Gen 10  : 0.9104
Gen 20  : 0.9042666666666667
Gen 30  : 0.9176
Gen 40  : 0.9176
Gen 50  : 0.9248
Gen 60  : 0.9226666666666666
Gen 70  : 0.9245333333333333
Gen 80  : 0.9242666666666667
Gen 90  : 0.9245333333333333


In [86]:
#TESTING
test_x = torch.from_numpy(x_test).to(torch.float32)
test_y = torch.from_numpy(y_test).to(torch.float32)
test = TensorDataset(test_x,test_y)
test = DataLoader(test,batch_size=1)

In [87]:
y_pred = model(test_x)
y_pred = torch.where(y_pred>=0.5, 1, 0).flatten()
accuracy = (y_pred == test_y).sum().float().item() / len(test.dataset)
print(accuracy)

0.8112


In [88]:
y_pred = best(test_x)
y_pred = torch.where(y_pred>=0.5, 1, 0).flatten()
ga_report=classification_report(y_pred,test_y)
print(ga_report)

              precision    recall  f1-score   support

           0       0.99      0.93      0.96      1207
           1       0.30      0.81      0.44        43

    accuracy                           0.93      1250
   macro avg       0.65      0.87      0.70      1250
weighted avg       0.97      0.93      0.94      1250



In [89]:
##ANT COLONY OPTIMIZATION

In [90]:
torch.manual_seed(6699)
torch.set_grad_enabled(False)

In [91]:
dummy = Model()
dummy_param = np.concatenate([i.numpy().flatten() for i in dummy.parameters()])
shape = [i.numpy().shape for i in model.parameters()]
size = [i[0]*i[1] if len(i) == 2 else i[0] for i in shape]
dim = len(dummy_param)
print("Dim : ", dim)
print("Layers Shape : ", shape)
print("Layers Size : ", size)
dummy = None
dummy_param = None

Dim :  209
Layers Shape :  [(16, 11), (16,), (1, 16), (1,)]
Layers Size :  [176, 16, 16, 1]


In [92]:
def model_to_vector(model):
    vector = np.concatenate([i.numpy().flatten() for i in model.parameters()])
    return vector
    
def vector_to_model(vector):
    param = list()
    cum_sum = 0
    for i in range(len(size)):
        array = vector[cum_sum : cum_sum + size[i]]
        array = array.reshape(shape[i])
        cum_sum += size[i]
        param.append(array)
    param = np.array(param, dtype="object")
    
    dummy_model = Model()
    for idx,wei in enumerate(dummy_model.parameters()):
        wei.data = (torch.tensor(param[idx])).to(torch.float32)
        
    return dummy_model    

def calc_accuracy(model):
    y_pred = model(train_x)
    y_pred = torch.where(y_pred>=0.5, 1, 0).flatten()
    acc = (y_pred == train_y).sum().float().item() / len(data.dataset)
    
    return acc

In [93]:
ants = 10
loops = 100
evaporation_rate = 0.2
influence_factor = 0.4

In [94]:
pheromones = np.ones(dim)
max_accuracy = 0
fittest_vector = None

for loop in range(loops):
    # Generate Solution
    paths = np.array([Model() for i in range(ants)])
    accuracy = []
    
    for ant in range(ants):
        vector = model_to_vector(paths[ant])
        vector = vector * pheromones
        model = vector_to_model(vector)
        acc = calc_accuracy(model)
        accuracy.append(acc)
        
        # Update the updated path
        paths[ant] = model

        model = None
        acc = None
    paths = paths[np.argsort(accuracy)]
    
    if accuracy[np.argmax(accuracy)] > max_accuracy:
        max_accuracy = accuracy[np.argmax(accuracy)]
        fittest_vector = model_to_vector(paths[-1])

    delta = 0
    for ant in range(ants):

        vector = model_to_vector(paths[ant])
        delta += (vector - fittest_vector)*influence_factor
        
    pheromones = (1-pheromones)*evaporation_rate + delta
    
    if loop%10 == 0:
        print("Iters {} :".format(loop), calc_accuracy(paths[-1]))

Iters 0 : 0.9016
Iters 10 : 0.9026666666666666
Iters 20 : 0.9026666666666666
Iters 30 : 0.9026666666666666
Iters 40 : 0.9029333333333334
Iters 50 : 0.9026666666666666
Iters 60 : 0.9026666666666666
Iters 70 : 0.9130666666666667
Iters 80 : 0.9026666666666666
Iters 90 : 0.9021333333333333


In [95]:
print("Maximum Accuracy : ", max_accuracy)
best_model = vector_to_model(fittest_vector)

Maximum Accuracy :  0.9232


In [96]:
#TESTING
y_pred = best_model(test_x)
y_pred = torch.where(y_pred>=0.5, 1, 0).flatten()
print(classification_report(y_pred,test_y))

              precision    recall  f1-score   support

           0       0.96      0.96      0.96      1143
           1       0.56      0.60      0.58       107

    accuracy                           0.92      1250
   macro avg       0.76      0.78      0.77      1250
weighted avg       0.93      0.92      0.93      1250



In [97]:
##PARTICLE SWARM OPTIMIZATION

In [98]:
from pyswarms.single import GlobalBestPSO
torch.manual_seed(420)
torch.set_grad_enabled(False)

model = Model()
param = np.concatenate([i.numpy().flatten() for i in model.parameters()])
shape = [i.numpy().shape for i in model.parameters()]
size = [i[0]*i[1] if len(i) == 2 else i[0] for i in shape]

print("Dim : ", len(param))
print("Layers Shape : ", shape)
print("Layers Size : ", size)

Dim :  209
Layers Shape :  [(16, 11), (16,), (1, 16), (1,)]
Layers Size :  [176, 16, 16, 1]


In [99]:
def objective_function(particle,shape=shape,size=size):
    accuracy = []
    output = []
    
    # Reshape the vector to weights and biases dimention
    for par in particle:
        param = list()
        cum_sum = 0
        for i in range(len(size)):
            array = par[cum_sum : cum_sum + size[i]]
            array = array.reshape(shape[i])
            cum_sum += size[i]
            param.append(array)
        param = np.array(param, dtype="object")
        output.append(param)
    
    for i in range(len(output)):
        # Copy Weights and Biases
        model = Model()
        for idx,wei in enumerate(model.parameters()):
            wei.data = (torch.tensor(output[i][idx])).to(torch.float32)
        
        # Calculate Accuracy
        y_pred = model(train_x)
        y_pred = torch.where(y_pred>=0.5, 1, 0).flatten()
        acc = (y_pred == train_y).sum().float().item() / len(data.dataset)
        accuracy.append(1 - acc) # Optimization function aims to reduce the cost so (1 - accuracy)
        
    return accuracy

In [100]:
options = {'c1': 0.6, 'c2': 0.3, 'w': 0.1}
dim = len(param)
x_max = 1.0 * np.ones(dim)
x_min = -1.0 * x_max
bounds = (x_min, x_max)

pso = GlobalBestPSO(n_particles=500, dimensions=209, options=options, bounds=bounds)

In [59]:
best_cost, best_parameters = pso.optimize(objective_function, iters=50)
print("Accuracy : ", 1 - best_cost)

2023-04-05 22:11:23,498 - pyswarms.single.global_best - INFO - Optimize for 50 iters with {'c1': 0.6, 'c2': 0.3, 'w': 0.1}
pyswarms.single.global_best: 100%|█████████████████████████████████████████████████████████████|50/50, best_cost=0.0493
2023-04-05 22:11:40,435 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.04933333333333334, best pos: [ 0.14848011  0.10349188  0.10447447 -0.15864278 -0.13296082  0.11275246
 -0.10220486 -0.29697417 -0.23323361 -0.28370793  0.58556348  0.11904705
  0.29336401  0.14168986  0.06249131  0.15813676  0.14734061  0.52107227
 -0.22412549  0.1179161   0.36226975  0.10651173  0.06556157  0.25010549
  0.02828201  0.18841723  0.12176045  0.38687897 -0.15035066  0.2662203
 -0.30492109 -0.38156719 -0.40896439 -0.03777549  0.18117844 -0.0849403
 -0.29019603  0.21881203  0.00367032 -0.08009338  0.43077075  0.03160016
 -0.04783791  0.43837156  0.0651608  -0.03085401 -0.05404157  0.69368672
 -0.1273383  -0.42550958  0.32619094 -0.290011

Accuracy :  0.9506666666666667


In [101]:
result = []
for par in [best_parameters]:
    param = list()
    cum_sum = 0
    for i in range(len(size)):
        array = par[cum_sum : cum_sum + size[i]]
        array = array.reshape(shape[i])
        cum_sum += size[i]
        param.append(array)
    param = np.array(param, dtype="object")
    result.append(param)

best_model = Model()
for idx,wei in enumerate(best_model.parameters()):
    wei.data = (torch.tensor(result[0][idx])).to(torch.float32)

# Calculate Accuracy
y_pred = best_model(train_x)
y_pred = torch.where(y_pred>=0.5, 1, 0).flatten()
acc = (y_pred == train_y).sum().float().item() / len(data.dataset)
print("Accuracy : ", acc)

Accuracy :  0.9506666666666667


In [102]:
#TESTING
y_pred = best_model(test_x)
y_pred = torch.where(y_pred>=0.5, 1, 0).flatten()
ps_report=classification_report(y_pred,test_y)
print(ps_report)

              precision    recall  f1-score   support

           0       0.98      0.96      0.97      1162
           1       0.57      0.74      0.64        88

    accuracy                           0.94      1250
   macro avg       0.77      0.85      0.80      1250
weighted avg       0.95      0.94      0.95      1250



In [111]:
#Cultural algorithm:

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from sklearn.metrics import accuracy_score


def fitness(individual):
    
    model.set_weights(individual)
    
    model.compile(optimizer=Adam(lr=0.01), loss='binary_crossentropy')
    model.fit(X_train, y_train, batch_size=10, epochs=100)
    
    y_pred = model.predict(X_test)
    y_pred = (y_pred > 0.5)
    return accuracy_score(y_test, y_pred)

def init():
    population = []
    for i in range(pop_size):
        weights = []
        for layer in model.layers:
            if len(layer.get_weights()) > 0:
                w = layer.get_weights()[0]
                b = layer.get_weights()[1]
                weights.append(w.flatten())
                weights.append(b.flatten())
        population.append(np.concatenate(weights))
    return population

def selection(population, fitness_values):
    sorted_indices = np.argsort(fitness_values)[::-1]
    selected_indices = sorted_indices[:int(pop_size/2)]
    return [population[i] for i in selected_indices]

def crossover(parent1, parent2):
    child1 = np.zeros_like(parent1)
    child2 = np.zeros_like(parent2)
    for i in range(len(parent1)):
        if np.random.rand() < p_c:
            child1[i] = parent2[i]
            child2[i] = parent1[i]
        else:
            child1[i] = parent1[i]
            child2[i] = parent2[i]
    return child1, child2

def mutation(child):
    for i in range(len(child)):
        if np.random.rand() < p_m:
            child[i] += np.random.uniform(-1, 1)
    return child

def cultural_algorithm():
    
    population = init()
    for gen in range(max_gen):
        
        fitness_values = [fitness(individual) for individual in population]
        
        parents = selection(population, fitness_values)
        
        offspring = []
        while len(offspring) < pop_size:
            parent1 = np.random.choice(parents)
            parent2 = np.random.choice(parents)
            child1, child2 = crossover(parent1, parent2)
            child1 = mutation(child1)
            child2 = mutation(child2)
            offspring.append(child1)
            offspring.append(child2)
        
        population = parents + offspring
        
        population = np.unique(population, axis=0)
        population = population[:pop_size]
   
    fitness_values = [fitness(individual) for individual in population]
    
    best_index = np.argmax(fitness_values)
    best_individual = population[best_index]
    model.set_weights(best_individual)
    
    model.compile(optimizer=Adam(lr=0.01), loss='binary_crossentropy')
    model.fit(X, y, batch_size=1, epochs=30)
    accuracy = model.evaluate(X_test, y_test)[1]
    print("Accuracy:", accuracy)

In [112]:
y_pred = best_model(test_x)
y_pred = torch.where(y_pred>=0.5, 1, 0).flatten()
ps_report=classification_report(y_pred,test_y)
print(ps_report)

              precision    recall  f1-score   support

           0       0.98      0.96      0.97      1162
           1       0.57      0.74      0.64        88

    accuracy                           0.94      1250
   macro avg       0.77      0.85      0.80      1250
weighted avg       0.95      0.94      0.95      1250

